Parse the images and create the corresponding Vectors

In [25]:
import os
import numpy as np
from PIL import Image

image_folder_path = 'dataset/Beagle'

# Get the list of all image files in the folder
image_files = [f for f in os.listdir(image_folder_path) if os.path.isfile(os.path.join(image_folder_path, f))]

# Function to load, resize, and preprocess an image
def load_and_preprocess_image(image_path, target_size=(64, 64)):
    image = Image.open(image_path).convert('L')  # Convert to grayscale for simplicity
    image = image.resize(target_size)            # Resize to the target size
    features = np.array(image).flatten()         # Flatten the image array to a 1D feature vector
    return features

# Load and preprocess all images
all_features = []
for image_file in image_files:
    image_path = os.path.join(image_folder_path, image_file)
    features = load_and_preprocess_image(image_path)
    all_features.append(features)

feature_matrix = np.array(all_features).T

print('Feature matrix shape:', feature_matrix)
print('Shape: ', feature_matrix.shape)

Feature matrix shape: [[  2 255 186 ...  88  83 255]
 [ 10 255 157 ...  89  87 255]
 [ 20 255  89 ...  84  89 255]
 ...
 [132 255 165 ... 123 121 255]
 [128 255 161 ... 105 118 255]
 [127 255 155 ...  64  96 255]]
Shape:  (4096, 100)
